<a href="https://colab.research.google.com/github/LongNguyen1984/TimeSeriesWithPython/blob/main/DateDataTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet pytorch-lightning==1.2.5


     |████████████████████████████████| 829kB 9.7MB/s 
     |████████████████████████████████| 276kB 29.4MB/s 
     |████████████████████████████████| 112kB 21.4MB/s 
     |████████████████████████████████| 276kB 29.4MB/s 
     |████████████████████████████████| 829kB 38.0MB/s 
     |████████████████████████████████| 1.3MB 44.9MB/s 
     |████████████████████████████████| 296kB 47.2MB/s 
     |████████████████████████████████| 143kB 52.3MB/s 


In [ ]:
!pip install --quiet tqdm==4.59.0

     |████████████████████████████████| 81kB 5.5MB/s 


In [ ]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

tqdm.pandas()

In [ ]:
pl.seed_everything(42)

Global seed set to 42


42

##Load Data

In [ ]:
!gdown --id 174pzX55XaNzWgjZSgX6Us13fnBvPCca6

Downloading...
From: https://drive.google.com/uc?id=174pzX55XaNzWgjZSgX6Us13fnBvPCca6
To: /content/Binance_BTCUSDT_minute.csv
37.2MB [00:00, 117MB/s] 


In [ ]:
from datetime import datetime
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
#custom_date_parser = lambda x: datetime.strptime(x, "%Y-%d-%m %H:%M")

In [ ]:
#df = pd.read_csv('Binance_BTCUSDT_minute.csv', parse_dates=['date'] ,date_parser=custom_date_parser, d) ##Not working because error in data
df = pd.read_csv('Binance_BTCUSDT_minute.csv')

In [ ]:
df =df.sort_values(by='date').reset_index(drop=True)
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,NaN,11784.67,11791.1,211.128,2489507.533,688.0,NaN,NaN,NaN,NaN
1,1.567970e+12,2019-09-08 17:57,BTC/USDT,10000.000,10000.000,10000.0,10000.0,0.001,10.0,1.0
2,1.567970e+12,2019-09-08 17:58,BTC/USDT,10000.000,10000.000,10000.0,10000.0,0.000,0.0,0.0
3,1.567970e+12,2019-09-08 17:59,BTC/USDT,10000.000,10000.000,10000.0,10000.0,0.001,10.0,1.0
4,1.567970e+12,2019-09-08 18:00,BTC/USDT,10000.000,10000.000,10000.0,10000.0,0.000,0.0,0.0


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(753594, 10)

####Convert to datetime

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

#### Cheking type of data

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753594 entries, 1 to 753594
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   unix         753594 non-null  float64       
 1   date         753594 non-null  datetime64[ns]
 2   symbol       753594 non-null  object        
 3   open         753594 non-null  float64       
 4   high         753594 non-null  float64       
 5   low          753594 non-null  float64       
 6   close        753594 non-null  float64       
 7   Volume BTC   753594 non-null  float64       
 8   Volume USDT  753594 non-null  float64       
 9   tradecount   753594 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 63.2+ MB


##Processing

In [ ]:
df.close

1         10000.00
2         10000.00
3         10000.00
4         10000.00
5         10000.00
            ...   
753590    57816.93
753591    57930.28
753592    58049.58
753593    58215.14
753594    58331.16
Name: close, Length: 753594, dtype: float64

In [ ]:
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
1,1.567970e+12,2019-09-08 17:57:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.001,10.0,1.0
2,1.567970e+12,2019-09-08 17:58:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.000,0.0,0.0
3,1.567970e+12,2019-09-08 17:59:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.001,10.0,1.0
4,1.567970e+12,2019-09-08 18:00:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.000,0.0,0.0
5,1.567970e+12,2019-09-08 18:01:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.000,0.0,0.0


In [ ]:
df['prev_close'] = df.shift(1)['close']

In [ ]:
df['close_change'] = df.progress_apply(
    lambda row: 0 if np.isnan(row.prev_close) else row.close - row.prev_close,
    axis=1
)

  0%|          | 0/753594 [00:00<?, ?it/s]

In [ ]:
df.head(100)

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount,prev_close,close_change
1,1.567970e+12,2019-09-08 17:57:00,BTC/USDT,10000.00,10000.00,10000.00,10000.00,0.001,10.0000,1.0,NaN,0.00
2,1.567970e+12,2019-09-08 17:58:00,BTC/USDT,10000.00,10000.00,10000.00,10000.00,0.000,0.0000,0.0,10000.00,0.00
3,1.567970e+12,2019-09-08 17:59:00,BTC/USDT,10000.00,10000.00,10000.00,10000.00,0.001,10.0000,1.0,10000.00,0.00
4,1.567970e+12,2019-09-08 18:00:00,BTC/USDT,10000.00,10000.00,10000.00,10000.00,0.000,0.0000,0.0,10000.00,0.00
5,1.567970e+12,2019-09-08 18:01:00,BTC/USDT,10000.00,10000.00,10000.00,10000.00,0.000,0.0000,0.0,10000.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
96,1.567970e+12,2019-09-08 19:33:00,BTC/USDT,10354.64,10354.99,10353.14,10353.14,11.456,118610.5884,12.0,10354.64,-1.50
97,1.567970e+12,2019-09-08 19:34:00,BTC/USDT,10353.14,10353.14,10344.33,10346.00,10.915,112954.9996,15.0,10353.14,-7.14
98,1.567970e+12,2019-09-08 19:35:00,BTC/USDT,10345.27,10347.09,10345.27,10347.09,10.965,113449.7966,12.0,10346.00,1.09
99,1.567970e+12,2019-09-08 19:36:00,BTC/USDT,10347.09,10347.09,10346.18,10346.18,11.528,119279.2229,15.0,10347.09,-0.91


In [ ]:
rows = []

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
  row_data = dict(
      day_of_week = row.date.dayofweek,
      day_of_month = row.date.day,
      week_of_year=row.date.week,
      month=row.date.month,
      open=row.open,
      high=row.high,
      low=row.low,
      close_change=row.close_change,
      close=row.close
  )
  rows.append(row_data)

features_df =pd.DataFrame(rows)

  0%|          | 0/753594 [00:00<?, ?it/s]

In [ ]:
features_df.shape

(753594, 9)

In [ ]:
features_df.head()

,day_of_week,day_of_month,week_of_year,month,open,high,low,close_change,close
0,6,8,36,9,10000.0,10000.0,10000.0,0.0,10000.0
1,6,8,36,9,10000.0,10000.0,10000.0,0.0,10000.0
2,6,8,36,9,10000.0,10000.0,10000.0,0.0,10000.0
3,6,8,36,9,10000.0,10000.0,10000.0,0.0,10000.0
4,6,8,36,9,10000.0,10000.0,10000.0,0.0,10000.0


In [ ]:
train_size = int(len(features_df)*0.9)
train_size

678234

In [ ]:
train_df, test_df = features_df[:train_size], features_df[train_size + 1:]
train_df.shape, test_df.shape

((678234, 9), (75359, 9))

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler = scaler.fit(train_df)

In [ ]:
train_df = pd.DataFrame(
    scaler.transform(train_df), 
    index=train_df.index,
    columns=train_df.columns)

In [ ]:
test_df = pd.DataFrame(
    scaler.transform(test_df), 
    index=test_df.index,
    columns=test_df.columns)

In [ ]:
train_df.head()

,day_of_week,day_of_month,week_of_year,month,open,high,low,close_change,close
0,1.0,-0.533333,0.346154,0.454545,-0.670418,-0.674046,-0.665888,-0.08716,-0.670418
1,1.0,-0.533333,0.346154,0.454545,-0.670418,-0.674046,-0.665888,-0.08716,-0.670418
2,1.0,-0.533333,0.346154,0.454545,-0.670418,-0.674046,-0.665888,-0.08716,-0.670418
3,1.0,-0.533333,0.346154,0.454545,-0.670418,-0.674046,-0.665888,-0.08716,-0.670418
4,1.0,-0.533333,0.346154,0.454545,-0.670418,-0.674046,-0.665888,-0.08716,-0.670418


In [ ]:
def create_sequences(input_data: pd.DataFrame, target_column, sequence_length):
  sequences = []
  data_size = len(input_data)

  for i in tqdm(range(data_size - sequence_length)):

    sequence = input_data[i: i + sequence_length]

    label_position = i + sequence_length
    label = input_data.iloc[label_position][target_column]

    sequences.append((sequence, label))
  return sequences

In [ ]:
sample_data = pd.DataFrame(dict(
    feature_1 = [1, 2, 3, 4, 5],
    label = [6, 7, 8, 9, 10]
))

sample_data.head()

,feature_1,label
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [ ]:
sample_sequences = create_sequences(sample_data, "label", sequence_length=3)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
len(sample_sequences)

2

In [ ]:
print(sample_sequences[0][0])
print()
print(f"label: {sample_sequences[0][1]}")

   feature_1  label
0          1      6
1          2      7
2          3      8

label: 9


In [ ]:
print(sample_sequences[1][0])
print()
print(f"label: {sample_sequences[1][1]}")

   feature_1  label
1          2      7
2          3      8
3          4      9

label: 10


In [ ]:
SEQUENCE_LENGTH =60

train_sequences = create_sequences(train_df, "close", SEQUENCE_LENGTH)
test_sequences = create_sequences(test_df, "close", SEQUENCE_LENGTH)

  0%|          | 0/678174 [00:00<?, ?it/s]

  0%|          | 0/75299 [00:00<?, ?it/s]

In [ ]:
train_sequences[0][0].shape

(60, 9)

In [ ]:
len(train_sequences)

678174

In [ ]:
len(train_df) - len(train_sequences) == SEQUENCE_LENGTH

True